<a href="https://colab.research.google.com/github/MengOonLee/Deep_learning/blob/master/TensorFlow2/Start/SequentialAPI/Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to TensorFlow 2


## Coding tutorials

#### [1. Hello TensorFlow!](#coding_tutorial_1)

<a id='coding_tutorial_1'></a>
## Hello TensorFlow!

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

ds_train = tf.data.Dataset.from_tensor_slices(tensors=(x_train, y_train))
ds_train = ds_train.map(map_func=lambda x, y:
    (tf.cast(x=x, dtype=tf.float32)/255., y))
ds_train = ds_train.map(map_func=lambda x, y:
    (x[..., tf.newaxis], y))
print("Train element spec:", ds_train.element_spec)

ds_test = tf.data.Dataset.from_tensor_slices(tensors=(x_test, y_test))
ds_test = ds_test.map(map_func=lambda x, y:
    (tf.cast(x=x, dtype=tf.float32)/255., y))
ds_test = ds_test.map(map_func=lambda x, y: (
    x[..., tf.newaxis], y))
print("Test element spec:", ds_test.element_spec)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Train element spec: (TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))
Test element spec: (TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.keras.utils.set_random_seed(seed=42)
tf.get_logger().setLevel("ERROR")

class MnistModel(tf.keras.Model):
    def __init__(self, num_classes, **kwargs):
        super().__init__(**kwargs)
        self.features_extractor = FeaturesExtractor()
        self.classifier = Classifier()

    def call(self, inputs, training=False):
        h = self.features_extractor(
            inputs=inputs, training=training)
        return h

class FeaturesExtractor(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flatten = tf.keras.layers.Flatten()

    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3),
            padding="same", activation=None, input_shape=input_shape)

    def call(self, inputs, training=False):
        h = self.conv(inputs=inputs)
        h = self.batchnorm(inputs=h, training=training)
        h = tf.nn.relu(features=h)
        h = self.flatten(inputs=h)
        return h

class Classifier(tf.keras.layers.Layer):
    

In [ ]:
import numpy as np
np.random.seed(seed=42)

# Load MNIST dataset
print("Loading train data...\n")
data = np.loadtxt(fname="data/mnist.csv", delimiter=",")
print("MNIST dataset loaded.\n")

x_train = data[:, 1:]
x_train = x_train / 255.
y_train = data[:, 0]

print("x train data size: {}".format(x_train.shape))
print("y train data size: {}".format(y_train.shape))

In [ ]:
# Import Tensorflow
import tensorflow as tf
tf.keras.utils.set_random_seed(42)
# Check its version & devices
print(tf.__version__)
print(tf.config.list_physical_devices())

# Train a feedforward neural network for image classification
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(
    units=16, activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.Dense(
    units=10, activation=tf.keras.activations.softmax
))
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
print("Training model...\n")
history = model.fit(
    x=x_train, y=y_train, validation_split=0.1,
    epochs=10, batch_size=32, verbose=0
)
print("Model trained successfully")

import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

df_history = pd.DataFrame(data=history.history, index=history.epoch)
metrics = ["loss", "sparse_categorical_accuracy"]
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
for ax, metric in zip(axes, metrics):
    sns.lineplot(ax=ax, data=df_history, label="train",
        x=df_history.index+1, y=metric
    )
    try:
        sns.lineplot(ax=ax, data=df_history, label="valid",
            x=df_history.index+1, y=f"val_{metric}"
        )
    except Exception:
        pass
    ax.legend(loc="best")
    ax.set_xlabel(xlabel="epoch")
    ax.set_ylabel(ylabel=f"{metric}")
    ax.set_title(label=f"{metric} vs. epoch")
fig.tight_layout()
plt.show()